# Official stations data

In this notebook, we will assemble a time series of meteorological measurements from meteorological stations in the study region. We will follow these steps:
1. Given a target year, use the Agrometeo API to select the temporal range of interest as the hottest heatwave, defined as follows: the hottest (based on mean temperature) period of at least 3 consecutive days with an average temperature over 27$^{\circ}$C (namely, the hottest period of level 4 warning days [according to the heat warning level definitions by MeteoSwiss](https://www.meteoswiss.admin.ch/weather/weather-and-climate-from-a-to-z/heat-warnings.html)).
2. Given the temporal range of interest obtained above, download the time series of meteorological measurements from the other official meteorological stations in the study region.
3. Resample all data to hourly resolution, assemble into a single long data frame and dump into a file

In [ ]:
import datetime

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from meteora import utils as meteora_utils
from meteora.clients import AgrometeoClient
from shapely import geometry

from uhi_cws_lausanne import official_stations_utils, plot_utils

figwidth, figheight = plt.rcParams["figure.figsize"]

HEATWAVE_N_CONSECUTIVE_DAYS = 3
HEATWAVE_THRESHOLD = 25

In [ ]:
# spatial extent
agglom_extent_filepath = "../data/raw/agglom-extent.gpkg"

# files to write
dst_ts_df_filepath = "../data/interim/official-ts-df.csv"
dst_stations_gdf_filepath = "../data/interim/official-stations.gpkg"

# select study period
start_year = 2021
end_year = 2023
# whether we want to include the days before and after the heatwave
days_before = 1
days_after = 1
# months to consider when querying the Agrometeo API
start_month = 5
end_month = 9

# S3 data
bucket_name = "ceat-data"
idaweb_key = "meteoswiss/idaweb/lausanne-agglom-2022-2023.txt"
vaudair_key = "vaud-air/VaudAir_AggloLausanne_20220101-20231231_20240613.xlsx"

# station locations
station_location_filepath = (
    "https://zenodo.org/record/4384675/files/station-locations.csv"
)
station_location_crs = "epsg:2056"

# date format for heatwave labels
STRFMT = "%Y/%m/%d"

## 1. Select study period

We will use the Agrometeo API to select the hottest heatwave period as described above.

In [ ]:
# select only land extent (agglom_extent_filepath has two geometries: land and lake)
region = gpd.read_file(agglom_extent_filepath)["geometry"].iloc[:1]

# download data
agrometeo_client = AgrometeoClient(region=region)
agrometeo_ts_df = pd.concat(
    [
        agrometeo_client.get_ts_df(
            "temperature",
            datetime.date(year, start_month, 1),
            datetime.date(year, end_month, 30),
            scale="hour",
        )
        for year in range(start_year, end_year + 1)
    ],
    axis="rows",
)
# convert to wide form
agrometeo_ts_df = meteora_utils.long_to_wide(agrometeo_ts_df)

# find consecutive days above threshold
day_mean_ts_ser = (
    agrometeo_ts_df.groupby(agrometeo_ts_df.index.date).mean().mean(axis="columns")
)
idx = (day_mean_ts_ser >= HEATWAVE_THRESHOLD).rolling(
    window=HEATWAVE_N_CONSECUTIVE_DAYS, center=True
).sum() >= HEATWAVE_N_CONSECUTIVE_DAYS
idx = idx | idx.shift(1) | idx.shift(-1)

heatwave_max_ser = pd.concat(
    [
        pd.Series(day_mean_ts_ser.loc[g[g].index].mean(), index=g.index)
        for i, g in idx.groupby(idx.ne(idx.shift()).cumsum())
        if g.any()
    ]
)
date_ser = pd.to_datetime(heatwave_max_ser.rename_axis("time").reset_index()["time"])
heatwave_range_df = date_ser.groupby(date_ser.diff().dt.days.ne(1).cumsum()).agg(
    start="first", end="last"
)
# add a day in the end to ensure that we get the data from the start of the first day to
# the end of the last day
heatwave_range_df["end"] = heatwave_range_df["end"] + datetime.timedelta(days=1)
# show the data frame
heatwave_range_df

## 2. Get data from other official stations

Download the data from a private S3 bucket.

In [ ]:
client = official_stations_utils.SpacesClient(bucket_name=bucket_name)

### 2.1 IDAWEB

In [ ]:
idaweb_ts_df = client.get_idaweb_df(idaweb_key)
idaweb_ts_df

### 2.2 Vaud'air

In [ ]:
# select the first four columns (the other ones are humidity)
vaudair_ts_df = client.get_vaudair_df(vaudair_key).iloc[:, :4]
vaudair_ts_df

## 3. Filter heatwaves period, resample and assemble

In [ ]:
official_ts_df = pd.concat(
    [
        pd.concat(
            [
                ts_df.loc[heatwave_start:heatwave_end]
                for heatwave_start, heatwave_end in heatwave_range_df.itertuples(
                    index=False
                )
            ],
            axis="rows",
        )
        .resample("H")
        .mean()
        .rename_axis("station", axis="columns")
        .stack()
        .rename("temperature")
        .reset_index()
        for ts_df in [agrometeo_ts_df, idaweb_ts_df, vaudair_ts_df]
    ]
)

# convert to wide format
official_ts_df = official_ts_df.pivot_table(
    index="time", columns="station", values="temperature"
)
# add heatwave id as outermost index
official_ts_df = (
    pd.concat(
        [
            official_ts_df.loc[start:end].assign(
                **{
                    "heatwave": "-".join(
                        [
                            f"{date.strftime(STRFMT)}"
                            for date in [start, end - datetime.timedelta(days=1)]
                        ]
                    )
                }
            )
            for start, end in heatwave_range_df.itertuples(index=False)
        ]
    )
    .reset_index()
    .set_index(["heatwave", "time"])
)
official_ts_df.head()

Let us now plot, for each heatwave, the hourly temperature averaged over stations and days of the heatwave:

In [ ]:
fig, ax = plt.subplots()

for heatwave, heatwave_ts_df in official_ts_df.groupby(level="heatwave"):
    heatwave_ts_df = heatwave_ts_df.stack(dropna=True).reset_index(name="T")
    sns.lineplot(
        heatwave_ts_df.assign(**{"hour": heatwave_ts_df["time"].dt.hour}),
        x="hour",
        y="T",
        ax=ax,
        label=heatwave,
    )

In [ ]:
fig, ax = plt.subplots()

for heatwave, heatwave_ts_df in official_ts_df.apply(
    lambda row: row - min(row), axis="columns"
).groupby(level="heatwave"):
    heatwave_ts_df = heatwave_ts_df.stack(dropna=True).reset_index(name="UHI")
    sns.lineplot(
        heatwave_ts_df.assign(**{"hour": heatwave_ts_df["time"].dt.hour}),
        x="hour",
        y="UHI",
        ax=ax,
        label=heatwave,
    )

## 4. Station locations

In [ ]:
official_stations_gser = official_stations_utils.get_station_gser(
    station_location_filepath, station_location_crs
)
# add missing station locations by hand (TODO: change UGLY hardcoded lines)
official_stations_gser["Nabel_Lausanne"] = geometry.Point(2538690, 1152615)
# official_stations_gser["WSLLAB"] = geometry.Point(2545761, 1160617)
# set CRS lost in line above
official_stations_gser = official_stations_gser.set_crs(station_location_crs)
# filter to keep only stations in our agglomeration extent
official_stations_gser = official_stations_gser[
    official_stations_gser.intersects(region.iloc[0])
]

In [ ]:
# plot by average temperature
plot_kws = {"legend_kwds": {"shrink": 0.5, "label": "T$_{mean}$ [$\degree$C]"}}
plot_utils.plot_stations_by_var(
    official_stations_gser, official_ts_df.mean().rename("T_mean"), plot_kws=plot_kws
)

## 4. Filter data and dump to files

In [ ]:
# stations to keep: stations from the time series data and within the extent
official_stations = official_stations_gser.index.intersection(official_ts_df.columns)
# filter time series data from stations of the region only
official_ts_df[official_stations].to_csv(dst_ts_df_filepath)
# filter to keep only stations in our time series data frame
official_stations_gser.loc[official_stations].to_file(dst_stations_gdf_filepath)